In [203]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta

In [205]:
# Function to fetch stock data
def fetch_data(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')
    frames = []
    for ticker in tickers:
        stock_data = data[ticker].dropna()
        stock_data['Ticker'] = ticker
        frames.append(stock_data)
    combined_data = pd.concat(frames)
    combined_data.index.names = ['Date']
    combined_data.reset_index(inplace=True)
    multi_index_df = combined_data.set_index(['Date', 'Ticker'])
    return multi_index_df

In [207]:
# Function to add technical indicators
def add_technical_indicators_with_look_forward(data):
    data = data.reset_index()
    indicators = []
    for ticker, group in data.groupby("Ticker"):
        group.set_index("Date", inplace=True)
        group['rsi'] = ta.rsi(group['Close'], length=14)
        group['momentum'] = ta.mom(group['Close'], length=10)
        group['sma_50'] = ta.sma(group['Close'], length=50)
        group['future_pct_change_21d'] = group['Close'].pct_change(21).shift(-21)
        indicators.append(group)
    enhanced_data = pd.concat(indicators)
    return enhanced_data.reset_index().set_index(['Date', 'Ticker'])

In [209]:
# Function to train models and predict
def train_xgboost_per_stock_with_lookforward(data):
    data = data.reset_index()
    stock_predictions = {}
    stock_models = {}
    for ticker, group in data.groupby("Ticker"):
        print(f"Training model for stock: {ticker}")
        group = group.dropna()
        X = group[['rsi', 'momentum', 'sma_50']]
        y = group['future_pct_change_21d']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        model = XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        predictions_df = pd.DataFrame({
            'Date': group.loc[X_test.index, 'Date'],
            'Predicted': y_pred,
            'Actual': y_test.values
        }).set_index('Date')
        stock_predictions[ticker] = predictions_df
        stock_models[ticker] = {'model': model, 'mse': mse}
    return stock_predictions, stock_models

In [211]:
# Function to calculate MSE for each stock
def calculate_mse_for_all_stocks(stock_predictions):
    mse_results = [{'Ticker': ticker, 'MSE': mean_squared_error(predictions['Actual'], predictions['Predicted'])}
                   for ticker, predictions in stock_predictions.items()]
    return pd.DataFrame(mse_results).sort_values(by='MSE', ascending=True)

In [213]:
# Function to predict future returns
def predict_future_returns(stock_models, data, days_ahead=21):
    future_predictions = []
    for ticker, model_info in stock_models.items():
        model = model_info['model']
        stock_data = pd.DataFrame(data).xs(ticker, level='Ticker').tail(1)
        latest_features = stock_data[['rsi', 'momentum', 'sma_50']].values
        future_return = model.predict(latest_features)
        future_predictions.append({
            'Ticker': ticker,
            'Predicted_Future_Return': future_return[0],
            'Prediction_Date': stock_data.index[0] + timedelta(days=days_ahead)
        })
    return pd.DataFrame(future_predictions).sort_values(by='Predicted_Future_Return', ascending=False)

In [215]:
df = pd.read_csv('stock_list.csv')
stocks_list = df['Stocks'].tolist()

# Fetch and process data
data = fetch_data(stocks_list, '2024-08-01', '2024-12-05')
enhanced_data = add_technical_indicators_with_look_forward(data)

# Train models and get predictions
stock_predictions, stock_models = train_xgboost_per_stock_with_lookforward(enhanced_data)

# Rank models by MSE
mse_df = calculate_mse_for_all_stocks(stock_predictions)
print("Model MSE Rankings:")
print(mse_df)

# Predict future returns
future_predictions_df = predict_future_returns(stock_models, enhanced_data)
print("\nFuture Predictions (21 Days Ahead):")
print(future_predictions_df)

[*********************100%***********************]  282 of 282 completed

1 Failed download:
['BRKB']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


Training model for stock: A
Training model for stock: AAPL
Training model for stock: ABBV
Training model for stock: ACGL
Training model for stock: ADBE
Training model for stock: ADI
Training model for stock: ADP
Training model for stock: ADSK
Training model for stock: AFL
Training model for stock: AIZ
Training model for stock: AJG
Training model for stock: AKAM
Training model for stock: ALNY
Training model for stock: AMAT
Training model for stock: AMD
Training model for stock: AMGN
Training model for stock: AMP
Training model for stock: AMT
Training model for stock: AMZN
Training model for stock: ANET
Training model for stock: ANSS
Training model for stock: AON
Training model for stock: APD
Training model for stock: AVGO
Training model for stock: AVTR
Training model for stock: AXON
Training model for stock: AXP
Training model for stock: BAC
Training model for stock: BALL
Training model for stock: BBY
Training model for stock: BG
Training model for stock: BK
Training model for stock: BK

### Method 1: Calculating Sharpe Ratio Without Using the Predictions
What is the Sharpe Ratio?
The Sharpe ratio compares the return of an investment with its risk. It is usually between -3 and 3 for a stock. The higher the sharpe ratio is, the better, as it means that there is limited risk and a good chance of making profit. 

To calculate the sharpe ratio, 3 pieces of information must be known:
1) Expected Return of a stock
2) Risk Free Rate (theoretical rate of return on an investment with zero risk) - often assumed to be 4%
3) Standard Deviation of a Stock's Return

Then, to get the sharpe ratio, we calculate ((step 1 - step 2) / step 3) 

With this method I used the following values for the above parameters:
1) Expected Return of a stock -> I calculated the average daily return of a stock in a year
2) Risk Free Rate -> Assumed to be 4%
3) Standard Deviation of a Stock's Return -> Took the standard deviation of one years historical data. Why? Standard deviation measures the risk or variability of returns which is what we want the denominator of the sharpe ratio to be.

As you can see, there is no need to use the previous predictions to calculate the sharpe ratio

In [217]:
import numpy as np

def calculate_sharpe_ratio(ticker, risk_free_rate=0.04):
    """
    Calculate the annualized Sharpe Ratio for a given stock ticker.
    """
    try:
        # Fetch one-year historical data
        data = yf.download(ticker, period='1y')
        
        # Check if data is available
        if data.empty:
            return None
        
        # Calculate daily returns
        data['Daily_Returns'] = data['Adj Close'].pct_change()
        
        # Calculate mean and standard deviation of daily returns
        mean_daily_return = data['Daily_Returns'].mean()
        std_daily_return = data['Daily_Returns'].std()
        
        # Check for zero standard deviation
        if std_daily_return == 0 or pd.isna(std_daily_return):
            return None
        
        # Annualize mean return and standard deviation
        mean_annual_return = mean_daily_return * 252
        std_annual_return = std_daily_return * np.sqrt(252)
        
        # Calculate Sharpe Ratio
        sharpe_ratio = (mean_annual_return - risk_free_rate) / std_annual_return
        return sharpe_ratio
    except Exception as e:
        print(f"Error calculating Sharpe Ratio for {ticker}: {e}")
        return None

# Load stock tickers from CSV
df = pd.read_csv('stock_list.csv')
tickers = df['Stocks'].tolist()

# Risk-free rate (annualized)
risk_free_rate = 0.04  # 4% annualized

# Calculate Sharpe Ratios
results = []
for ticker in tickers:
    print(f"Processing {ticker}...")
    sharpe = calculate_sharpe_ratio(ticker, risk_free_rate)
    results.append({'Ticker': ticker, 'Sharpe_Ratio': sharpe})

# Save results to a DataFrame and export to CSV
results_df = pd.DataFrame(results)
results_df.to_csv(r"sharpe_ratios.csv", index=False)

print("Sharpe Ratios calculated and saved to 'sharpe_ratios.csv'.")
results_df

[*********************100%***********************]  1 of 1 completed

Processing AAPL...
Processing NVDA...



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Processing MSFT...
Processing AMZN...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed


Processing LDOS...


[*********************100%***********************]  1 of 1 completed

Sharpe Ratios calculated and saved to 'sharpe_ratios.csv'.


,Ticker,Sharpe_Ratio
0,AAPL,1.051276
1,NVDA,2.290208
2,MSFT,0.874652
3,AMZN,1.590617
4,GOOG,1.043272
...,...,...
277,ALNY,0.959634
278,HCA,0.790680
279,NTNX,0.896458
280,JNPR,1.011063


In [219]:
results_df

,Ticker,Sharpe_Ratio
0,AAPL,1.051276
1,NVDA,2.290208
2,MSFT,0.874652
3,AMZN,1.590617
4,GOOG,1.043272
...,...,...
277,ALNY,0.959634
278,HCA,0.790680
279,NTNX,0.896458
280,JNPR,1.011063


### Method 2: Calculating Sharpe Ratio Using the Predictions we Calculated
With this method I used the following values for the parameters:
1) Expected Return of a stock -> For this, our predictions for 21 days in the future were used. 
2) Risk Free Rate -> Assumed to be 4% annualy. Was then scaled for 21 days accordingly (since 4% is the annual rate). 
3) Standard Deviation of a Stock's Return -> Took the standard deviation of one years historical data scaled to 21 days 

I think this method produces worse results then the first one because we are using our own predictinos... so yeah, if we use our predictions our results could be worse. 

In [222]:
predictions = future_predictions_df.iloc[:, :2]
predictions

risk_free_rate = 0.03 
prediction_horizon = 21  

# Get historical data to calculate volatility
def get_historical_volatility(ticker):
    stock_data = yf.download(ticker, period="1y", interval="1d")
    stock_data['Daily_Return'] = stock_data['Adj Close'].pct_change()
    daily_volatility = stock_data['Daily_Return'].std()
    return daily_volatility * np.sqrt(prediction_horizon)  # Scale to prediction horizon

# Calculate volatility for each stock
predictions['Volatility'] = predictions['Ticker'].apply(get_historical_volatility)

# Calculate Sharpe Ratio
predictions['Sharpe_Ratio'] = (predictions['Predicted_Future_Return'] - risk_free_rate / 252 * prediction_horizon) / predictions['Volatility']

# Display the results
print(predictions[['Ticker', 'Predicted_Future_Return', 'Volatility', 'Sharpe_Ratio']])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

    Ticker  Predicted_Future_Return  Volatility  Sharpe_Ratio
188   MSTR                 0.626532    0.312685      1.995723
216   PLTR                 0.470428    0.183767      2.546312
60    COIN                 0.427522    0.251599      1.689279
25    AXON                 0.375689    0.128910      2.894962
251   TSLA                 0.355134    0.178714      1.973176
..     ...                      ...         ...           ...
12    ALNY                -0.164347    0.144824     -1.152069
137    IQV                -0.173185    0.087123     -2.016525
24    AVTR                -0.175689    0.088840     -2.005734
183   MRNA                -0.220080    0.176203     -1.263203
228   REGN                -0.229010    0.061668     -3.754113

[281 rows x 4 columns]


In [225]:
predictions[['Ticker', 'Predicted_Future_Return', 'Volatility', 'Sharpe_Ratio']]

,Ticker,Predicted_Future_Return,Volatility,Sharpe_Ratio
188,MSTR,0.626532,0.312685,1.995723
216,PLTR,0.470428,0.183767,2.546312
60,COIN,0.427522,0.251599,1.689279
25,AXON,0.375689,0.128910,2.894962
251,TSLA,0.355134,0.178714,1.973176
...,...,...,...,...
12,ALNY,-0.164347,0.144824,-1.152069
137,IQV,-0.173185,0.087123,-2.016525
24,AVTR,-0.175689,0.088840,-2.005734
183,MRNA,-0.220080,0.176203,-1.263203
